<a href="https://colab.research.google.com/github/eliastam/Covid19-Machine-Learning-Project/blob/main/COVIDSTH_Revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn
import time
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from datetime import date

%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)

# Task 1: Acquire, preprocess, and analyze the data

In [2]:
# LOAD AND PREPROCESS SYMPTOMS DATAFRAME
max_nans_ratio = 0.35
symdf = pd.read_csv('data/2020_US_weekly_symptoms_dataset.csv')
# drop columns with all "nan"s in column
symdf = symdf.dropna(1, how='all')
# drop columns with too many "nan"s
symdf = symdf.loc[:, (symdf.isnull().sum(axis=0) / len(symdf.index)) <= max_nans_ratio]

In [3]:
# LOAD AND PREPROCESS HOSPITALIZATION DATAFRAME
colslist = ['open_covid_region_code', 'region_name', 'date', 'hospitalized_new']
ccdf = pd.read_csv('data/aggregated_cc_by.csv', usecols=colslist)
# drop columns that don't relate to the US
ccdf = ccdf[ccdf.open_covid_region_code.str.contains("US-")]
# convert from daily data to weekly
ccdf['date'] = ccdf['date'].astype('datetime64[ns]')
ccdf = ccdf.groupby("open_covid_region_code").resample('W-Mon', label='right', closed='right', on='date').sum().reset_index()
ccdf['date'] = ccdf['date'].astype('str')
# sort dataframe
ccdf = ccdf.sort_values(['open_covid_region_code','date'], ascending = [False, True])

In [4]:
# CREATE MERGED DATAFRAME
mergeddf = symdf.merge(ccdf, how='inner', left_on=['open_covid_region_code','date'], right_on=['open_covid_region_code','date'])

In [5]:
# Fill missing values
df = mergeddf
print(df.shape)
df = df.fillna(df.mean())
df.to_csv('cleaned_data.csv')

(465, 16)


# Task 2: Visualize and cluster the data

# 2.1

In [6]:
# FIND POPULAR SYMPTOMS
N = 5
# get popularity for non-string columns
symcols = symdf.select_dtypes(exclude=object).columns
popularity = list(zip(symcols, df[symcols].sum()))
# sort from most to least popular
popularity = sorted(popularity, key=lambda x: x[1], reverse=True)
# top N popular columns
popularsyms = [p[0] for p in popularity[:N]]

#print(popularity[:N])
#print(list(df.columns))

In [7]:
for sym in popularsyms: 
    fig = px.scatter(df, x="date", y=sym, range_y=[0,90], color="open_covid_region_code", title=sym)
    fig.show()

# 2.2

In [8]:
# Convert to numpy
searchdf = df.select_dtypes(exclude=object).drop("hospitalized_new", 1)
X = searchdf.to_numpy()
print(X.shape)
print(X[0])
print(list(searchdf.columns))

(465, 9)
[12.47062295  7.91       11.48954693  9.76       11.67782178 12.45
  7.07        8.08       10.97639498]
['symptom:Angular cheilitis', 'symptom:Aphonia', 'symptom:Crackles', 'symptom:Dysautonomia', 'symptom:Hemolysis', 'symptom:Laryngitis', 'symptom:Rectal pain', 'symptom:Shallow breathing', 'symptom:Ventricular fibrillation']


In [9]:
# Standardize inputs
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

# PCA
pca = PCA(n_components=2)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)

In [10]:
fig = px.scatter(x=X_pca[:,0], y=X_pca[:,1], color=df["sub_region_1"], labels={'x':'PC1', 'y':'PC2'})
fig.show()

# 2.3

In [11]:
kmeans = KMeans(n_clusters=15, random_state=0).fit(X_scaled)
y = kmeans.predict(X_scaled)
title = {'text': "kmeans on original data", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}
fig = px.scatter(x=X_pca[:,0], y=X_pca[:,1], color=y, labels={'x':'PC1', 'y':'PC2'})
fig.update_layout(title=title)
fig.show()

In [12]:
kmeans_pca = KMeans(n_clusters=15, random_state=0).fit(X_pca)
y = kmeans_pca.predict(X_pca)
title = {'text': "kmeans on lower dimensional data", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}
fig = px.scatter(x=X_pca[:,0], y=X_pca[:,1], color=y, labels={'x':'PC1', 'y':'PC2'})
fig.update_layout(title=title)
fig.show()

# Task 3: Supervised Learning

## 3.1

## 3.1.A. Region Split

In [13]:
def region_split(df, train_size = 0.8, split_seed = None):
    regions = df['sub_region_1'].unique()
    if split_seed is None:
        np.random.shuffle(regions)
    else:
        rng = np.random.RandomState(split_seed)
        rng.shuffle(regions)
    train_ind = int(train_size * len(regions))
    train_regions, val_regions = regions[:train_ind], regions[train_ind:]

    # create train  split
    train_data = df.loc[df['sub_region_1'].isin(train_regions)].drop('date', 1).sample(frac=1)
    train_regions = train_data.sub_region_1.tolist()
    train_data = train_data.select_dtypes(exclude=object)
    X_train, y_train = train_data.drop("hospitalized_new", 1).to_numpy(), train_data["hospitalized_new"].to_numpy()

    # create val split
    val_data = df.loc[df['sub_region_1'].isin(val_regions)].drop('date', 1).sample(frac=1)
    val_regions = val_data.sub_region_1.tolist()
    val_data = val_data.select_dtypes(exclude=object)
    X_val, y_val = val_data.drop("hospitalized_new", 1).to_numpy(), val_data["hospitalized_new"].to_numpy()
    return X_train, y_train, X_val, y_val, train_regions, val_regions

X_train, y_train, X_val, y_val, train_regions, val_regions = region_split(df, split_seed=5)
print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"X_val.shape: {X_val.shape}")
print(f"y_val.shape: {y_val.shape}")
print(f"set(train_regions): {set(train_regions)}")
print(f"set(val_regions): {set(val_regions)}")
print(f"len(train_regions): {len(train_regions)}")
print(f"len(val_regions): {len(val_regions)}")
print(f"train_regions[:10]: {train_regions[:10]}")
print(f"val_regions[:10]: {val_regions[:10]}")

X_train.shape: (349, 9)
y_train.shape: (349,)
X_val.shape: (116, 9)
y_val.shape: (116,)
set(train_regions): {'Delaware', 'New Mexico', 'Maine', 'Alaska', 'Rhode Island', 'Nebraska', 'Idaho', 'North Dakota', 'District of Columbia', 'New Hampshire', 'Wyoming', 'South Dakota'}
set(val_regions): {'Hawaii', 'Vermont', 'Montana', 'West Virginia'}
len(train_regions): 349
len(val_regions): 116
train_regions[:10]: ['New Hampshire', 'South Dakota', 'Nebraska', 'Delaware', 'District of Columbia', 'Maine', 'Idaho', 'Maine', 'New Mexico', 'Alaska']
val_regions[:10]: ['Montana', 'Hawaii', 'Hawaii', 'West Virginia', 'Vermont', 'Vermont', 'Montana', 'Hawaii', 'West Virginia', 'Montana']


## 3.1.B. Date Split

In [14]:
def date_split(df, split_date = None, train_size = 0.8, return_regions=False):
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index(df['date'])
    df = df.sort_index()

    if split_date is None:
        num_train = int(train_size * len(df))
        split_date = df.iloc[num_train].date

    train_date = (df.iloc[0].date, split_date)
    val_date = (split_date, df.iloc[-1].date)

    # create train  split
    train_data = df[:split_date].sample(frac=1)
    train_times = train_data.date.astype('str').tolist()
    if return_regions:
        regions_train = train_data.sub_region_1.to_numpy()
    train_data = train_data.select_dtypes(exclude=object).drop("date", 1)
    X_train, y_train = train_data.drop("hospitalized_new", 1).to_numpy(), train_data["hospitalized_new"].to_numpy()

    # create val split
    val_data = df[split_date:].sample(frac=1)
    val_times = val_data.date.astype('str').tolist()
    if return_regions:
        regions_val = val_data.sub_region_1.to_numpy()
    val_data = val_data.select_dtypes(exclude=object).drop("date", 1)
    X_val, y_val = val_data.drop("hospitalized_new", 1).to_numpy(), val_data["hospitalized_new"].to_numpy()

    if return_regions:
        return X_train, y_train, X_val, y_val, regions_train, regions_val

    return X_train, y_train, X_val, y_val, train_times, val_times

X_train, y_train, X_val, y_val, train_times, val_times = date_split(df, split_date="2020-08-10")
print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"X_val.shape: {X_val.shape}")
print(f"y_val.shape: {y_val.shape}")
print(f"max(train_times): {max(train_times)}")
print(f"min(val_times): {min(val_times)}")
print(f"train_times[:10]: {train_times[:10]}")
print(f"val_times[:10]: {val_times[:10]}")

X_train.shape: (369, 9)
y_train.shape: (369,)
X_val.shape: (112, 9)
y_val.shape: (112,)
max(train_times): 2020-08-10
min(val_times): 2020-08-10
train_times[:10]: ['2020-05-11', '2020-07-27', '2020-08-03', '2020-05-18', '2020-03-23', '2020-03-23', '2020-05-11', '2020-05-18', '2020-07-27', '2020-04-27']
val_times[:10]: ['2020-09-21', '2020-08-31', '2020-09-21', '2020-08-10', '2020-08-31', '2020-08-17', '2020-08-24', '2020-08-31', '2020-09-14', '2020-09-14']


## 3.2


## 3.2.A. Find best `k` for KNN

In [15]:
knn_time = time.time()
k_values = np.arange(1, 40)
k_mse = np.zeros_like(k_values).astype(float)
for i, k in enumerate(k_values):
    N_val = 0
    for fold in range(5):
        X_train, y_train, X_val, y_val, *_ = region_split(df, split_seed=fold)

        # Standardize inputs
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        model = KNeighborsRegressor(n_neighbors=k).fit(X_train_scaled, y_train)
        y_val_pred = model.predict(X_val_scaled)
        N_val += len(y)
        k_mse[i] += np.sum((y_val - y_val_pred) ** 2)
    k_mse[i] /= N_val


In [16]:
best_k = k_values[np.argmin(k_mse)]
print(f"Achieved min mse of {np.min(k_mse)} with k={best_k}.")
print(f"Knn training time: {time.time() - knn_time}")

Achieved min mse of 1698.5094593787337 with k=12.
Knn training time: 1.827117681503296


In [17]:
k_mse_pd = pd.DataFrame(data=np.vstack([k_values, k_mse]).T, index=np.arange(k_values.shape[0]), columns=['k', 'mse'])
fig = px.line(k_mse_pd, x="k", y="mse", title='KNN validation MSE (region split)')
fig.show()

## 3.2.B. Find Best `max_depth` for DT

In [18]:
dt_time = time.time()
d_values = np.arange(2, 10)
d_mse = np.zeros_like(d_values).astype(float)
for i, d in enumerate(d_values):
    N_val = 0
    for fold in range(5):
        X_train, y_train, X_val, y_val, *_ = region_split(df, split_seed=fold)

        # Standardize inputs
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        model = DecisionTreeRegressor(max_depth=d).fit(X_train_scaled, y_train)
        y_val_pred = model.predict(X_val_scaled)
        N_val += len(y)
        d_mse[i] += np.sum((y_val - y_val_pred) ** 2)
    d_mse[i] /= N_val

In [19]:
best_depth = d_values[np.argmin(d_mse)]
print(f"Achieved min mse of {np.min(d_mse)} with max_depth={best_depth}.")
print(f"DT training time: {time.time() - dt_time}")

Achieved min mse of 1916.7671204984922 with max_depth=5.
DT training time: 0.36003613471984863


In [20]:
d_mse_pd = pd.DataFrame(data=np.vstack([d_values, d_mse]).T, index=np.arange(d_values.shape[0]), columns=['max_depth', 'mse'])
fig = px.line(d_mse_pd, x="max_depth", y="mse", title='DT validation MSE (region split)')
fig.show()

## 3.2.C. Region split training

In [21]:
N_val = 0
DT_mse = 0
KNN_mse = 0
for fold in range(5):
    X_train, y_train, X_val, y_val, *_ = region_split(df, split_seed=fold + 10)

    # Standardize inputs
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # DT
    DT_model = DecisionTreeRegressor(max_depth=best_depth).fit(X_train_scaled, y_train)
    y_val_pred = DT_model.predict(X_val_scaled)
    N_val += len(y)
    DT_mse += np.sum((y_val - y_val_pred) ** 2)

    # KNN
    KNN_model = KNeighborsRegressor(n_neighbors=best_k).fit(X_train_scaled, y_train)
    y_val_pred = KNN_model.predict(X_val_scaled)
    N_val += len(y)
    KNN_mse += np.sum((y_val - y_val_pred) ** 2)

DT_mse /= N_val
KNN_mse /= N_val
print("Region split")
print(f"Decision Tree validation MSE: {DT_mse}")
print(f"KNN validation MSE: {KNN_mse}")

Region split
Decision Tree validation MSE: 736.9606494224337
KNN validation MSE: 639.5121296296295


## 3.2.D. Date split training

In [22]:
N_val = 0
DT_mse = 0
KNN_mse = 0
for fold in range(5):
    X_train, y_train, X_val, y_val, *_ = date_split(df, split_date="2020-08-10")

    # Standardize inputs
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # DT
    DT_model = DecisionTreeRegressor(max_depth=best_depth).fit(X_train_scaled, y_train)
    y_val_pred = DT_model.predict(X_val_scaled)
    N_val += len(y)
    DT_mse += np.sum((y_val - y_val_pred) ** 2)

    # KNN
    KNN_model = KNeighborsRegressor(n_neighbors=best_k).fit(X_train_scaled, y_train)
    y_val_pred = KNN_model.predict(X_val_scaled)
    N_val += len(y)
    KNN_mse += np.sum((y_val - y_val_pred) ** 2)

DT_mse /= N_val
KNN_mse /= N_val
print("Date split")
print(f"Decision Tree validation MSE: {DT_mse}")
print(f"KNN validation MSE: {KNN_mse}")

Date split
Decision Tree validation MSE: 286.16754194014663
KNN validation MSE: 140.29374253285542


## 3.3

### 3.3.1 Train a model for each cluster

In [23]:
N_train = 0
N_val = 0
DT_mse_train = 0
DT_mse_val = 0
for fold in range(5):
    X_train, y_train, X_val, y_val, *_ = date_split(df, split_date="2020-08-10")

    # Standardize inputs
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    train_clusters = kmeans.predict(X_train_scaled)
    val_clusters = kmeans.predict(X_val_scaled)
    for i in range(kmeans.n_clusters):
        X_train_i = X_train_scaled[train_clusters == i]
        y_train_i = y_train[train_clusters == i]
        X_val_i = X_val_scaled[val_clusters == i]
        y_val_i = y_val[val_clusters == i]
        N_train += len(y_train_i)
        N_val += len(y_val_i)
        DT_model = DecisionTreeRegressor(max_depth=best_depth).fit(X_train_i, y_train_i)
        DT_mse_train += np.sum((y_train_i - DT_model.predict(X_train_i)) ** 2)
        if len(X_val_i) != 0:
            DT_mse_val += np.sum((y_val_i - DT_model.predict(X_val_i)) ** 2)

DT_mse_train /= N_train
DT_mse_val /= N_val

In [24]:
print("15 models (1 for each kmeans cluster) results:")
print(f"DT mse: train {DT_mse_train}, val {DT_mse_val}")

15 models (1 for each kmeans cluster) results:
DT mse: train 356.4233922972982, val 1894.2287743706704


### 3.3.2 Train a model for each region

In [25]:
N_train = 0
N_val = 0
DT_mse_train = 0
DT_mse_val = 0
for fold in range(5):
    X_train, y_train, X_val, y_val, regions_train, regions_val = date_split(df, split_date="2020-08-10", return_regions=True)

    # Standardize inputs
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    for region in set(regions_train):
        X_train_i = X_train_scaled[regions_train == region]
        y_train_i = y_train[regions_train == region]
        X_val_i = X_val_scaled[regions_val == region]
        y_val_i = y_val[regions_val == region]
        N_train += len(y_train_i)
        N_val += len(y_val_i)
        DT_model = DecisionTreeRegressor(max_depth=best_depth).fit(X_train_i, y_train_i)
        DT_mse_train += np.sum((y_train_i - DT_model.predict(X_train_i)) ** 2)
        if len(X_val_i) != 0:
            DT_mse_val += np.sum((y_val_i - DT_model.predict(X_val_i)) ** 2)

DT_mse_train /= N_train
DT_mse_val /= N_val

In [26]:
print("15 models (1 for each region) results:")
print(f"DT mse: train {DT_mse_train}, val {DT_mse_val}")

15 models (1 for each region) results:
DT mse: train 97.56623757904244, val 1949.5263523065473


## BONUS

Compare classifications on 5 features, where the 5 features are created/extracted using the following 3 different schemes:

1. Use the top 5 principal component features
2. Use the top 5 features with the highest (absolute) correlation with hospitalization
3. Use the features corresponding to the 5 largest (absolute) weights of a linear regression model trained on the data

In [27]:
# do PCA and use top 5 features
# correlation between hospitalization and each region and keep top 5 features
# linear regression and only keep top 5 features
pca = PCA(n_components=5)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)

corr = df.corr().drop('hospitalized_new')
symptopms = corr.nlargest(5, 'hospitalized_new').index
print(symptopms)

y =  df['hospitalized_new'].to_numpy()
reg = LinearRegression()
reg.fit(X, y)
inds = np.argsort(reg.coef_)
print(symcols[inds[:5]])
#print(inds)
#print(searchdf.iloc[inds])
searchdf = df.select_dtypes(exclude=object).drop("hospitalized_new", 1)

Index(['symptom:Aphonia', 'symptom:Rectal pain', 'symptom:Angular cheilitis',
       'symptom:Ventricular fibrillation', 'symptom:Crackles'],
      dtype='object')
Index(['symptom:Laryngitis', 'symptom:Dysautonomia',
       'symptom:Ventricular fibrillation', 'symptom:Hemolysis',
       'symptom:Shallow breathing'],
      dtype='object')
